In [ ]:
import os
import re
import csv
from glob import iglob
from pathlib import Path
import numpy as np

In [ ]:
BASE_DIR = "/docker/data/ksb/TestDir/"
DATA_BASE_DIR = os.path.join(BASE_DIR, "articles")

ORIGIN_PATH = os.path.join(DATA_BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(DATA_BASE_DIR,"Pretty-Data")
SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Summary-Preprocessed-Data")
SWORDS_PATH = os.path.join(DATA_BASE_DIR, "StopWordList.txt")
MODEL_PATH = os.path.join(BASE_DIR, "Word-Embedding-Model")

In [ ]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


In [ ]:
file_name = "SentencePiece_train.txt"
result = []

for idx, proc_article_path in enumerate(iglob(os.path.join(PREPROCESSED_PATH, '**.csv'), recursive=False)):
    
    f_proc= open(proc_article_path, 'r', newline="\n", encoding="utf-8")
    for [idx, title, contents] in csv.reader(f_proc):
        if contents is '': continue

        cont_list = contents.split("\t")
        result.append('\n'.join(cont_list))
    f_proc.close()

with open(file_name, 'w', encoding='utf-8') as f:
    f.write('\n'.join(result))

In [ ]:
templates= '--input={} \
--pad_id={} \
--bos_id={} \
--eos_id={} \
--unk_id={} \
--model_prefix={} \
--vocab_size={} \
--character_coverage={} \
--model_type={}'

In [ ]:
pad_id=0  
vocab_size = 70000 
model_num = len(list(iglob('**.vocab', recursive=False)))
prefix = 'spm-{}'.format(model_num) 
bos_id=1
eos_id=2
unk_id=3
character_coverage = 1.0
model_type ='word' 

In [ ]:
cmd = templates.format(file_name,
                pad_id,
                bos_id,
                eos_id,
                unk_id,
                prefix,
                vocab_size,
                character_coverage,
                model_type)
cmd

'--input=SentencePiece_train.txt --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3 --model_prefix=spm-2 --vocab_size=70000 --character_coverage=1.0 --model_type=word'

In [ ]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train(cmd)